<a href="https://colab.research.google.com/github/norayehia/Machine-learning-secuirty-data/blob/main/finalaics_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install category_encoders

     |████████████████████████████████| 82 kB 388 kB/s 


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
import pandas as pd
# importing required libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import category_encoders as ce
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/aics/assinmen1/traindata.csv') 
df.head()

,ID,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,level,Class
0,0,0,tcp,http,SF,295,2584,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,3,0.0,0.0,0.0,0.0,1.0,0.0,0.67,101,255,1.00,0.00,0.01,0.03,0.0,0.0,0.00,0.01,21,0
1,1,0,tcp,http,SF,312,839,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,7,0.0,0.0,0.0,0.0,1.0,0.0,0.29,223,255,1.00,0.00,0.00,0.02,0.0,0.0,0.01,0.01,21,0
2,2,0,tcp,ftp,SF,1225,2451,0,0,0,28,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.00,255,116,0.45,0.02,0.00,0.00,0.0,0.0,0.00,0.00,11,1
3,3,0,tcp,http,SF,293,296,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,4,0.0,0.0,0.0,0.0,1.0,0.0,0.50,216,255,1.00,0.00,0.00,0.02,0.0,0.0,0.00,0.02,21,0
4,4,0,tcp,X11,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0.0,0.0,0.5,1.0,0.5,1.0,0.00,115,1,0.01,0.03,0.01,0.00,0.0,0.0,0.01,1.00,11,0


https://stackoverflow.com/questions/24458645/label-encoding-across-multiple-columns-in-scikit-learn

In [ ]:
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [ ]:

# encode categorical variables with ordinal encoding
#encoder = ce.OrdinalEncoder(cols=['ID', 'duration', 'protocol_type', 'service', 'flag', 'src_bytes'])
#X= encoder.fit_transform(X)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder


"""
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit_transform(df)
"""

df.apply(LabelEncoder().fit_transform)
#or
encoding_pipeline = Pipeline([
    ('encoding',MultiColumnLabelEncoder(columns=[ 'ID', 'duration', 'protocol_type', 'service', 'flag', 'src_bytes']))
    # add more pipeline steps as needed
])
encoding_pipeline.fit_transform(df)
#or
from sklearn.preprocessing import OneHotEncoder

OneHotEncoder(handle_unknown='ignore').fit_transform(df)


#housing = pd.read_csv("/content/sample_data/california_housing_train.csv")
#scaler = preprocessing.MinMaxScaler()
#names =X.columns
#d = scaler.fit_transform(X)
#X = pd.DataFrame(d, columns=names)
#X.head()


In [ ]:
df=df.apply(LabelEncoder().fit_transform)

In [ ]:
df

,ID,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,level,Class
0,0,0,1,22,9,270,1691,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,96,0,50,101,255,100,0,1,3,0,0,0,1,21,0
1,1,0,1,22,9,287,697,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,6,0,0,0,0,96,0,29,223,255,100,0,0,2,0,0,1,1,21,0
2,2,0,1,18,9,830,1632,0,0,0,17,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,96,0,0,255,116,45,2,0,0,0,0,0,0,11,1
3,3,0,1,22,9,268,236,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,3,0,0,0,0,96,0,44,216,255,100,0,0,2,0,0,0,2,21,0
4,4,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,35,39,50,77,0,115,1,1,3,1,0,0,0,1,99,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22997,22997,0,1,22,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,70,39,96,0,0,4,227,100,0,25,15,0,0,100,86,21,0
22998,22998,0,1,57,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,205,12,69,51,0,0,6,5,0,255,13,5,5,0,0,99,86,0,0,19,1
22999,22999,0,1,46,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,121,1,69,51,0,0,2,6,0,255,16,6,8,0,0,99,86,0,0,21,1
23000,23000,0,2,41,9,413,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,269,267,0,0,0,0,96,0,0,255,255,100,0,100,0,0,0,0,0,17,0


In [ ]:
X = df.drop(['Class'], axis=1)
y = df['Class']

In [ ]:
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

In [ ]:
corr_features = correlation(X, 0.7)
len(set(corr_features))

13

In [ ]:
X=X.drop(corr_features,axis=1)

In [ ]:
X.shape

(23002, 30)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
rfc=RandomForestClassifier(random_state=42)

In [ ]:
param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

In [ ]:
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(x_train, y_train)

In [ ]:
CV_rfc.best_params_

In [ ]:
rfc1=RandomForestClassifier(random_state=42, max_features='auto', n_estimators= 500, max_depth=9, criterion='gini')

In [ ]:
#rfc1.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=9, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
predran=rfc1.predict(x_test)

In [ ]:
confusion_matrix(y_test, predran)

array([[3603,    6],
       [   6, 3286]])

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predran))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3609
           1       1.00      1.00      1.00      3292

    accuracy                           1.00      6901
   macro avg       1.00      1.00      1.00      6901
weighted avg       1.00      1.00      1.00      6901



the same but on all 

In [ ]:
rfc2=RandomForestClassifier(random_state=42, max_features='auto', n_estimators= 500, max_depth=9, criterion='gini')

In [ ]:
rfc2.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=9, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
X.shape

(23002, 30)

In [ ]:
X_test1 = pd.read_csv('/content/testdata.csv')

In [ ]:
#scaler = preprocessing.MinMaxScaler()
#names =X_test1.columns
##X_test1 = encoder.transform(X_test1)

X_test1=X_test1.apply(LabelEncoder().fit_transform)
#d = scaler.fit_transform(X_test1)
X_test1=X_test1.drop(corr_features,axis=1)
#X_test1 = pd.DataFrame(d, columns=names)
X_test1.head()

,ID,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,count,srv_count,serror_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,level
0,0,4,1,43,9,28,69,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,255,4,0,0,16
1,1,0,1,45,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,240,5,87,6,0,255,8,0,0,21
2,2,0,2,45,9,49,39,0,0,0,0,0,0,0,0,0,0,0,0,0,493,456,0,0,0,255,0,100,0,17
3,3,0,1,22,9,341,494,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,8,0,12,6,21
4,4,0,0,14,9,552,0,0,0,0,0,0,0,0,0,0,0,0,0,0,311,300,0,0,0,255,0,100,0,18


In [ ]:
X_test1.shape

(22501, 30)

In [ ]:
op_rf=rfc1.predict(X_test1)
import numpy as np
import pandas as pd
pd.DataFrame(op_rf, columns=['Class']).to_csv('op_rf.csv')

In [ ]:
op_rf2=rfc2.predict(X_test1)
import numpy as np
import pandas as pd
pd.DataFrame(op_rf2, columns=['Class']).to_csv('op_rf2.csv')

/////////////////////////////////////////////////////////////